In [12]:
from analysis import *
import matplotlib.pyplot as plt
import pickle
import os
import numpy as np
from bookkeeper import *

In [13]:
SIM_NAME = "selfish_mine_test"
SIM_REPS = 1
SIM_ITERS = 5000000

print(SIM_NAME)
for REP in range(SIM_REPS): 
    print(REP)
    location = f'../dumps/{SIM_NAME}_{REP}'
    nodes = []
    for filename in os.listdir(os.fsencode(location)):
        filename = filename.decode("utf-8")
        with open(f'{location}/{filename}', 'rb') as f:
            if filename ==  'bookkeeper':
                bookkeeper = pickle.load(f)
            else:
                nodes.append(pickle.load(f))

    an = Analysis(bookkeeper, nodes)
            
    print(f'Loaded {len(nodes)} nodes.')

    blocks = an.get_all_blocks()
    print(f'Total blocks:\t{len(blocks)}')

    stale_rates = [an.stale_block_rate(node) for node in nodes]
    print(f'Stale rate:\t{round(np.mean(stale_rates), 5)}')

    block_ints = [an.avg_block_interval(node) for node in nodes]
    print(f'Block interval:\t{np.mean(block_ints)} iters')

    # delays_50 = [an.block_percentile_delay(block, 0.5) for id, block in blocks.items()]
    # delays_90 = [an.block_percentile_delay(block, 0.9) for id, block in blocks.items()]
    # print(f'50% delay:\t{round(np.mean(delays_50), 2)} iters')
    # print(f'90% delay:\t{round(np.mean(delays_90), 2)} iters')

    # tps = an.transactions_per_second(blocks.values(), SIM_ITERS / 10)
    # print(f'Tx per second:\t{tps}')

selfish_mine_test
0
Loaded 2 nodes.
Total blocks:	833
Stale rate:	0.244
Block interval:	7950.020766773163 iters


In [14]:
honest = [node for node in nodes if 'HONEST' in node.name]
selfish = [node for node in nodes if 'SELFISH' in node.name]
print(len(honest))
print(len(selfish))

1
1


In [15]:
count = 0
for block in honest[0].blockchain.values():
    if type(block) != str and block.miner == 'SELFISH':
        count += 1

print(count)
print(len(honest[0].blockchain))

339
834


In [16]:
for block in honest[0].blockchain.values():
    print(block.height, block.miner, block.id, block.prev_id)

0 satoshi 36dbb9a5-2d62-441d-b154-c159fcf70eaa None
1 HONEST 550fa666-1b96-45ce-9641-d8582dd0b293 36dbb9a5-2d62-441d-b154-c159fcf70eaa
1 SELFISH [SELFISH]2cd1810b-fbf1-475c-a6f9-5782a8af214d 36dbb9a5-2d62-441d-b154-c159fcf70eaa
2 SELFISH [SELFISH]ecde275e-2667-4878-929d-c5f0a77e73f8 [SELFISH]2cd1810b-fbf1-475c-a6f9-5782a8af214d
3 HONEST cf362db0-8d33-4cdf-8a1f-09ab3925685d [SELFISH]ecde275e-2667-4878-929d-c5f0a77e73f8
3 SELFISH [SELFISH]4cd6e592-8050-4f82-85a8-af5f4ccfb2c1 [SELFISH]ecde275e-2667-4878-929d-c5f0a77e73f8
4 SELFISH [SELFISH]d63d7972-3221-402c-9f05-c31cf3f171d3 [SELFISH]4cd6e592-8050-4f82-85a8-af5f4ccfb2c1
5 SELFISH [SELFISH]e0de5846-8813-43d5-9093-6f46ec26a364 [SELFISH]d63d7972-3221-402c-9f05-c31cf3f171d3
6 HONEST e89657f7-b6e3-4819-b93b-0bebb078f96b [SELFISH]e0de5846-8813-43d5-9093-6f46ec26a364
7 HONEST f702f957-ba84-4748-ba6d-ce74c3fc68cd e89657f7-b6e3-4819-b93b-0bebb078f96b
8 HONEST c110765e-07b1-4f3b-865e-fa0ac2c7b01c f702f957-ba84-4748-ba6d-ce74c3fc68cd
8 SELFISH [SEL

In [7]:
main_chain = an.get_longest_chain(an.get_all_blocks())

In [8]:
h_count, s_count = 0, 0
for block in main_chain:
    if  block.miner == 'HONEST':
        h_count += 1
    elif block.miner == 'SELFISH':
        s_count += 1

print(h_count)
print(s_count)

516
7


In [9]:
node = selfish[0]
[block.height for id, block in node.private_chain.items()]


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [10]:
selfish[0].private_chain

{'d11f5d95-7470-46da-99c9-ef526cd71842': <bitcoin.models.BTCBlock at 0x7f9b74f0bbe0>,
 '5ec40aa3-5174-46e5-9c11-e0de9c4285e1': <bitcoin.models.BTCBlock at 0x7f9b74f0bd30>,
 '[SELFISH]acf9ae1f-036c-4cde-9f23-dd33bc242991': <bitcoin.models.BTCBlock at 0x7f9b74f0bf28>,
 '[SELFISH]6738ff65-2419-43bc-8a38-939e05e955f0': <bitcoin.models.BTCBlock at 0x7f9b74f0bf98>,
 '[SELFISH]35ae7073-5fd7-4c51-9e74-742330f64397': <bitcoin.models.BTCBlock at 0x7f9b74ec7128>,
 '[SELFISH]385290e4-d167-4492-ae6d-1d1aac7d1fc9': <bitcoin.models.BTCBlock at 0x7f9b74ec7208>,
 '[SELFISH]38995792-e893-4511-94d8-644fa8658d1d': <bitcoin.models.BTCBlock at 0x7f9b74ec7278>,
 '[SELFISH]cc3fd1ae-0d2a-4538-b8b1-8c9637ac47e6': <bitcoin.models.BTCBlock at 0x7f9b74ec72e8>,
 '[SELFISH]d4830e39-82f2-4f0c-af5d-89e2b3ef9345': <bitcoin.models.BTCBlock at 0x7f9b74ec7358>,
 '[SELFISH]fb1c7de6-8a25-4e36-a430-1fbdda011af0': <bitcoin.models.BTCBlock at 0x7f9b74ec76d8>,
 '[SELFISH]de7114a3-f7e5-48d9-91ea-16437dec6cb0': <bitcoin.models.BT

In [11]:
honest = 0
selfish = 0
for block in main_chain:
    if 'HONEST' in block.miner:
        honest += 1
    elif 'SELFISH' in block.miner:
        selfish += 1
print(f'HONEST: {honest}')
print(f'SELFISH: {selfish}')


HONEST: 516
SELFISH: 7
